In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("C:/Users/mario/OneDrive/Bureau/M1 Eco Stats/Python S2/Airbnb/Dataframefinal_clean.csv")

In [3]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Localisation', 'name', 'date', 'Emplacement', 'Localisation2'], axis=1)

In [4]:
df

,Nombre de chambres,Prix par nuit,Nombre de voyageurs,Nombre de lits,Nombre de salles de bains,Vue sur le jardin_True,Vue sur la vallée_True,Vue sur la montagne_True,Baignoire_True,Sèche-cheveux_True,...,Ascenseur_True,Climatisation_True,Piscine_True,Vue sur le lac_True,Vue sur la baie_True,Saison,Score,Score1,Score2,Logement codé
0,1.0,27.0,1.0,1.0,1.0,0,0,0,0,1,...,0,0,0,0,0,0.0,-0.037923,-0.281573,-0.721694,2.0
1,1.0,90.0,4.0,4.0,1.0,1,0,1,0,1,...,0,0,0,1,0,0.0,0.355693,0.240040,-0.096574,4.0
2,1.0,50.0,2.0,1.0,1.0,0,0,0,0,0,...,0,0,0,0,0,0.0,0.326580,0.096383,-0.038575,5.0
3,1.0,190.0,4.0,2.0,1.0,0,0,0,0,1,...,1,0,0,0,0,0.0,0.751613,0.599922,0.638247,4.0
4,1.0,85.0,4.0,2.0,1.0,0,1,1,0,0,...,0,0,0,1,0,0.0,0.074936,-0.252272,-0.610029,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,1.0,155.0,4.0,2.0,1.0,0,0,0,1,1,...,0,1,0,0,0,0.0,0.865991,0.680334,0.865991,4.0
1008,3.0,354.0,6.0,3.0,3.0,0,0,0,0,1,...,0,1,0,0,0,0.0,0.865991,0.680334,0.865991,4.0
1009,1.0,205.0,3.0,1.0,1.0,0,1,1,0,1,...,0,0,1,1,0,0.0,-0.037923,-0.281573,-0.721694,4.0
1010,1.0,120.0,3.0,3.0,1.0,0,0,0,1,1,...,0,0,0,0,0,0.0,0.838768,0.653977,0.701537,5.0


Our goal here is to develop a classification model that will allow us to identify housing types. To do this, we tested three classification algorithms, which we ranked according to the F1 score. The F1-score is a useful metric for evaluating the performance of classification models with 2 or more classes. It combines the precision and recall scores of a classifier into a single metric by taking their harmonic mean.

But first, we had to choose our most relevant explanatory variables through a Random Forest.

In [5]:
# We define X as the features set and y as the target set
X = df.drop(['Logement codé'],axis=1)
y = df['Logement codé']

# We display each variable's accuracy
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X, y)
imp_var = pd.Series(rf.feature_importances_, index = X.columns).sort_values(ascending = False)
imp_var

Prix par nuit                 0.125734
Nombre de voyageurs           0.074425
Score2                        0.065839
Score1                        0.065047
Score                         0.062624
Vaisselle et couverts_True    0.048161
Nombre de lits                0.040265
Four_True                     0.036725
Cuisine_True                  0.032442
Nombre de chambres            0.028307
Lave-linge_True               0.026084
Saison                        0.023944
Oreillers_True                0.021278
Climatisation_True            0.020153
TV_True                       0.020037
Vue sur le jardin_True        0.018742
Vue sur la montagne_True      0.018169
Bidet_True                    0.018126
Fer à repasser_True           0.017418
Vue sur le lac_True           0.017044
Nombre de salles de bains     0.015901
Livres_True                   0.015734
Baignoire_True                0.015516
Wifi_True                     0.015037
Chauffage_True                0.014872
Draps_True               

In [6]:
# We decide to keep only the 5 first variables, since we want to avoid overfitting issues
X = X[['Prix par nuit', 'Nombre de voyageurs', 'Vaisselle et couverts_True', 'Score']]

In [7]:
from sklearn.model_selection import train_test_split

# We split our dataset with a 60 % train set, and a 40 % test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, 
                                                    shuffle = True, stratify=y)

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.3, 
                                                    shuffle = True, stratify=y_test)

## Naive Bayes

The naive Bayes classifier is based on Bayes' theorem and is a classic of probability theory. It is based on conditional probabilities.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_nb = gnb.fit(X_train, y_train).predict(X_test)

from sklearn.metrics import f1_score
f1_score(y_test, y_pred_nb, average = 'weighted')

0.5558313590129615

## Support Vector Classification

The SVC algorithm is part of the Support Vector Machine family. An SVM is an algorithm that uses the vector component of the elements of the training dataset to determine a preferred orientation.

In [9]:
from sklearn import svm
C = 1.0 # regularization parameter
svc = svm.SVC(C=C)
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)

f1_score(y_test, y_pred_svc, average = 'weighted')

0.497625677657156

## K Nearest Neighbors

This algorithm allows to retain the most represented class among the k outputs associated with the k inputs closest to the new input x.

In [10]:
from sklearn.neighbors import KNeighborsClassifier
#instanciation et définition du k
knn = KNeighborsClassifier(n_neighbors = 3)
#training
knn.fit(X_train,y_train)

y_pred_knn = knn.predict(X_test)

f1_score(y_test, y_pred_knn, average = 'weighted')

0.5657913917068846

## Random Forest

The Random Forest algorithm is a classification algorithm that reduces the variance of the predictions of a single decision tree, thus improving its performance. To do this, it combines many decision trees in a bagging approach.

In [11]:
from sklearn.ensemble import RandomForestClassifier
rf_estimator = RandomForestClassifier()

In [12]:
# We fit the Random Forest on our train set
rf_estimator.fit(X_train, y_train)
# Then we predict our target based on X_train, the features train set, and we stock it in y_train_pred
y_train_pred_rf = rf_estimator.predict(X_train)

In [13]:
# We now fit the Random Forest on our test set
rf_estimator.fit(X_test, y_test)
# Then we predict our target based on X_test, the features test set, and we stock it in y_test_pred
y_test_pred_rf = rf_estimator.predict(X_test)

In [14]:
# We show our recall score on our test set
f1_score(y_test, y_test_pred_rf, average = 'weighted')

0.9758366133952989

In view of the results, we will choose to keep the model based on a Random Forest, with a F1-score of 97 %.